In [1]:
import joblib
import sys
import os
import warnings
import datetime as dt
warnings.filterwarnings("ignore")
project_root = os.path.abspath(os.path.join("..", "scraper"))
sys.path.insert(0, project_root)
from utils import *

In [2]:
# Connect to the AWS database
connection, cursor = database_connection_and_cursor("CSGO")

# Prepare the query with a parameter placeholder
query = """
    SELECT *
    FROM hltv_stats
    WHERE date > %s
"""

# Define the date you want to filter by
filter_date = '2024-01-01'

# Execute the query with the parameter
cursor.execute(query, (filter_date,))

# Fetch the results
cs_data = cursor.fetchall()

cols = [
    'ID', 'Event', 'Date', 'Map', 'Map Number', 'Team', 'Name',
    'Kills', 'Headshots', 'Assists', 'Deaths', 'Kast', 'K-D Diff',
    'ADR', 'FK Diff', 'Rating', 'Team Score', 'Opponent Score',
    'Teammate 1', 'Teammate 2', 'Teammate 3', 'Teammate 4', 'Opponent 1',
    'Opponent 2', 'Opponent 3', 'Opponent 4', 'Opponent 5', 'Teammate 5',
    'Opponent 6', 'Opponent 7', 'Teammate 6'
]
cs_data = pd.DataFrame(cs_data, columns=cols)

# Convert object statistic columns to float values
cs_data["Kast"] = cs_data["Kast"].astype("float")
cs_data["ADR"] = cs_data["ADR"].astype("float")
cs_data["Rating"] = cs_data["Rating"].astype("float")

# Unique teams and players
team_df = cs_data[["Date", "Name", "Team"]].drop_duplicates()
team_df.reset_index(drop=True, inplace=True)

# Concat the data
cs_data = \
cs_data.merge(
    right=team_df.add_prefix("Opponent "),
    left_on=["Date", "Opponent 1"],
    right_on=["Opponent Date", "Opponent Name"],
    how="left",
).drop(columns=["Opponent Date", "Opponent Name"])

# Opponents
opponents = cs_data.pop("Opponent Team")

# Single map column is just MAPS 1
cs_data["Map Number"] = cs_data["Map Number"].str.replace("Single Map", "1")

# Append back to the dataframe
cs_data.insert(5, "Opponent Team", opponents)
cs_stats = cs_data.iloc[:, :].query("(Map!='Best of 3') and (Map!='Best of 2') and (Map!= 'All')").reset_index(drop=True)

# Drop the columns
cs_stats_f = cs_stats.drop(columns=["ID", "K-D Diff", "FK Diff", "Team Score", "Opponent Score"])

# Filter the dataset to only MAPS 1, 2, and 3
cs_stats_f = cs_stats_f.groupby(by=["Event", "Date", "Team", "Opponent Team"]).filter(lambda group: set(group["Map Number"]).issubset({"1", "2", "3"}))
cs_stats_f.reset_index(drop=True, inplace=True)

In [3]:
def total_maps_statistics(df: pd.DataFrame, three_maps: bool=False) -> pd.DataFrame:
    """
    Totals up the statistics on the dataframe and returns a new dataframe

    Params:
        df (pd.DataFrame): The dataframe
        three_maps (bool): If it is three maps

    Returns:
        df (pd.DataFrame): New dataframe
    """
    df = df.copy()
    # Total up the numbers
    stat_columns = ["Kills", "Headshots", "Assists", "Deaths", "Kast", "ADR", "Rating"]
    # If statistics of three maps should be sum
    if three_maps:
        for stat_column in stat_columns:
            idx = cs_stats_f.columns.get_indexer([stat_column])[0]
            if stat_column in ["Kast", "ADR", "Rating"]:
                df.insert(loc=idx, column=stat_column, value=(df[f"{stat_column} Map 1"] + df[f"{stat_column} Map 2"] + df[f"{stat_column} Map 3"])/3)
            else:
                df.insert(loc=idx, column=stat_column, value=df[f"{stat_column} Map 1"] + df[f"{stat_column} Map 2"] + df[f"{stat_column} Map 3"])
        drop_cols = [f"{stat_column} Map 1" for stat_column in stat_columns] + [f"{stat_column} Map 2" for stat_column in stat_columns] + [f"{stat_column} Map 3" for stat_column in stat_columns]
        df.drop(columns=drop_cols, inplace=True)
        df["Map Number"] = "MAPS 1-3"
        return df
    # Only two maps
    else:
        for stat_column in stat_columns:
            idx = cs_stats_f.columns.get_indexer([stat_column])[0]
            if stat_column in ["Kast", "ADR", "Rating"]:
                df.insert(loc=idx, column=stat_column, value=(df[f"{stat_column} Map 1"] + df[f"{stat_column} Map 2"])/2)
            else:
                df.insert(loc=idx, column=stat_column, value=df[f"{stat_column} Map 1"] + df[f"{stat_column} Map 2"])
        drop_cols = [f"{stat_column} Map 1" for stat_column in stat_columns] + [f"{stat_column} Map 2" for stat_column in stat_columns]
        df.drop(columns=drop_cols, inplace=True)
        df["Map Number"] = "MAPS 1-2"
        return df

In [4]:
# Seperate the Maps to make it easier merge
map_1 = cs_stats_f[cs_stats_f["Map Number"] == "1"]
map_2 = cs_stats_f[cs_stats_f["Map Number"] == "2"]
map_3 = cs_stats_f[cs_stats_f["Map Number"] == "3"]

# Merge the dataframes to calculate for Maps 1-2
map_1_and_2 = pd.merge(
    left=map_1,
    right=map_2[
        ["Event", "Date", "Name", "Team", "Opponent Team", "Kills", "Headshots", "Assists", "Deaths", "Kast", "ADR", "Rating"]
        ],
    suffixes = (" Map 1", " Map 2"),
    on=["Event", "Date", "Name", "Team", "Opponent Team"],
)

# Merge the dataframe to calculate for Maps 1-3
map_1_and_2_and_3 = map_1_and_2.merge(
    right=map_3[
        ["Event", "Date", "Name", "Team", "Opponent Team", "Kills", "Headshots", "Assists", "Deaths", "Kast", "ADR", "Rating"]
    ],
    on=["Event", "Date", "Name", "Team", "Opponent Team"],
).rename(columns={
    "Kills": "Kills Map 3",
    "Headshots": "Headshots Map 3",
    "Assists": "Assists Map 3",
    "Deaths": "Deaths Map 3",
    "Kast": "Kast Map 3",
    "ADR": "ADR Map 3",
    "Rating": "Rating Map 3"
    })

# Apply the function
map_1_and_2 = total_maps_statistics(df=map_1_and_2)
map_1_and_2_and_3 = total_maps_statistics(df=map_1_and_2_and_3, three_maps=True)

# Change the Map Number column
map_1["Map Number"] = "MAPS 1"
map_3["Map Number"] = "MAPS 3"

# Exclude Map 2
cs_maps_and_stats = pd.concat(
    [
        map_1,
        map_3,
        map_1_and_2,
        map_1_and_2_and_3
    ],
    ignore_index=True
)


# Weighted moving average (weights)
weights = np.array([0.25, 0.20, 0.15, 0.125, 0.115, 0.10, 0.05, 0.01])

# Window size
n = 8

# Columns to apply weighted moving averages
weight_cols = ["Kills", "Headshots", "Assists", "Deaths", "Kast", "ADR", "Rating"]

# Encode the players by using their mean rating
player_ratings = cs_data.groupby("Name")["Rating"].agg(['mean', 'std'])

# Make dictionaries to encode names and map back to the name
player_mean_rating_dict = player_ratings['mean'].to_dict()
player_mean_rating_dict_reversed = {v: k for k, v in player_mean_rating_dict.items()}

# Encode the teams
team_ratings = cs_data.groupby(["Team",])["Rating"].agg(['mean', 'median', 'std'])

# Make dictionaries to encode teams and map back
team_std_rating_dict = team_ratings["std"].to_dict()
team_std_rating_dict_reversed = {v: k for k, v in team_std_rating_dict.items()}

In [5]:
model = joblib.load("xgr_model_v2.joblib")

def backtrack_projections(date: datetime, name: str, player_team: str, opp_team: str, map_type: str):
    model_inputs = {
    "WMA Kills": 0,
    "WMA Headshots": 0,
    "WMA Assists": 0,
    "WMA Deaths": 0,
    "WMA Kast": 0,
    "WMA ADR": 0,
    "WMA Rating": 0,
    "Map Number MAPS 1": 0,
    "Map Number MAPS 1-2": 0,
    "Map Number MAPS 1-3": 0,
    "Map Number MAPS 3": 0,
    "Team": 0,
    "Opponent Team": 0,
    "Name": ""
    }
    if "(Combo)" in map_type:
        return 0
    map_type = map_type.replace("MAPS", "MAP").replace("MAP", "MAPS")
    stat_type = map_type.split()[-1]
    map_type = " ".join(map_type.split()[:2])
    df = cs_maps_and_stats[
        (cs_maps_and_stats["Date"] < date) &
        (cs_maps_and_stats["Name"] == name) &
        (cs_maps_and_stats["Team"] == player_team) &
        (cs_maps_and_stats["Map Number"] == map_type)
    ]
    df_lower = cs_maps_and_stats[
        (cs_maps_and_stats["Date"] < date) &
        (cs_maps_and_stats["Name"] == name.lower()) &
        (cs_maps_and_stats["Team"] == player_team) &
        (cs_maps_and_stats["Map Number"] == map_type)
    ]
    if (len(df) <= 7) & (len(df_lower) <= 7):
        return 0
    elif len(df_lower) >= 8:
        df = df_lower
        name = name.lower()
    else:
        df = df
    dot_product = df[weight_cols].tail(n).apply(lambda group: np.dot(weights[::-1], group), raw=True)
    for col in dot_product.keys():
        column = f"WMA {col}"
        model_inputs[column] = dot_product.loc[col]
    model_inputs[f"Map Number {map_type}"] = 1
    # If the data cannot be mapped using the dictionaries
    try:
        model_inputs["Name"] = player_mean_rating_dict[name]
        model_inputs["Team"] = team_std_rating_dict[player_team]
        model_inputs["Opponent Team"] = team_std_rating_dict[opp_team]
    except KeyError:
        return 0
    inp = [np.array(list(model_inputs.values()))]
    prediction = model.predict(inp)[0]
    if stat_type == "Headshots":
        return prediction[1]
    return prediction[0]

In [6]:
# Execute the query to fetch all rows from the prizepicks_lines table
cursor.execute("""
    SELECT *
    FROM prizepicks_lines
""")

# Fetch all the results
results = cursor.fetchall()

cols = [
    'ID', 'Date', 'Time', "Stat Type",
    "Player Name", "Player Team", "Opp Team",
    "Line"
]
prizepicks = pd.DataFrame(results, columns=cols)
prizepicks.drop(columns=["ID", "Time"], inplace=True)
prizepicks = prizepicks.dropna()
prizepicks["Date"] = pd.to_datetime(prizepicks["Date"])
prizepicks["Stat Type"] = prizepicks["Stat Type"].str.replace("MAPS", "MAP").str.replace("MAP", "MAPS")

In [7]:
prizepicks["Model Prediction"] = prizepicks.apply(lambda col: backtrack_projections(col["Date"], col["Player Name"], col["Player Team"], col["Opp Team"], col["Stat Type"]), axis=1)
no_predictions = prizepicks[prizepicks["Model Prediction"] == 0]
prizepicks = prizepicks[prizepicks["Model Prediction"] != 0]
prizepicks.reset_index(drop=True, inplace=True)

In [8]:
def actual_result(date: datetime, name: str, opp_team: str, stat_type: str):
    game_date = date.date()
    if "Kills" in stat_type:
        stat_type_lis = stat_type.split("Kills")
        stat_name = "Kills"
    else:
        stat_type_lis = stat_type.split("Headshots")
        stat_name = "Headshots"
    map_name = stat_type_lis[0].strip()
    actual = cs_maps_and_stats[
    (cs_maps_and_stats["Date"].dt.date == game_date) &
    (cs_maps_and_stats["Name"] == name) &
    (cs_maps_and_stats["Opponent Team"] == opp_team) &
    (cs_maps_and_stats["Map Number"] == map_name)
    ]
    if len(actual) == 1:
        return actual.iloc[0][stat_name]
    else:
        return 0

In [9]:
actuals_nums = prizepicks.apply(lambda row: actual_result(row["Date"], row["Player Name"], row["Opp Team"], row["Stat Type"]), axis=1)

In [10]:
prizepicks.insert(6, "Actual", actuals_nums)

prizepicks_hit_rate_df = prizepicks[prizepicks["Actual"] != 0].reset_index(drop=True)

prizepicks_hit_rate_df["Result"] = np.where(
    (
        (prizepicks_hit_rate_df["Model Prediction"] <= prizepicks_hit_rate_df["Line"]) &
        (prizepicks_hit_rate_df["Actual"] <= prizepicks_hit_rate_df["Line"])
    )
    |
    (
        (prizepicks_hit_rate_df["Model Prediction"] >= prizepicks_hit_rate_df["Line"]) &
        (prizepicks_hit_rate_df["Actual"] >= prizepicks_hit_rate_df["Line"])
    ),
    "Correct",
    "Wrong"
)

prizepicks_hit_rate_df.to_csv("overall_hit_rate.csv")

In [11]:
def results_by_days(data: pd.DataFrame, n_days: int = 1, specific_day: bool = True):
    # Define target date
    target_day = (dt.datetime.today() - dt.timedelta(days=n_days)).date()
    data["Date"] = pd.to_datetime(data["Date"]).dt.date

    # Filter by specific day or range
    if (n_days == 1) | (specific_day):
        data = data[data["Date"] == target_day]
    else:
        data = data[data["Date"] >= target_day]

    # Define MAP level categorization based on 'Stat Type' column
    def map_level(stat_type):
        if "MAPS 1-2" in stat_type:
            return "MAPS 1-2"
        elif "MAPS 1-3" in stat_type:
            return "MAPS 1-3"
        elif "MAPS 1" in stat_type:
            return "MAPS 1"
        elif "MAPS 3" in stat_type:
            return "MAPS 3"
        else:
            return "Unknown"

    # Apply MAP level categorization
    data["MAP Level"] = data["Stat Type"].apply(map_level)

    # Initialize results container
    results_by_map = {}

    # Loop through each MAP level and calculate statistics
    for map_level in data["MAP Level"].unique():
        map_data = data[data["MAP Level"] == map_level]

        predicted_overs_and_unders = np.where(
            map_data["Model Prediction"] <= map_data["Line"], "Under", "Over"
        )
        actual_overs_and_unders = np.where(
            map_data["Actual"] <= map_data["Line"], "Under", "Over"
        )

        # Calculate correct Over and Under predictions
        correct_overs = np.sum((predicted_overs_and_unders == "Over") & (actual_overs_and_unders == "Over"))
        correct_unders = np.sum((predicted_overs_and_unders == "Under") & (actual_overs_and_unders == "Under"))

        # Predicted counts
        predicted_overs = np.sum(predicted_overs_and_unders == "Over")
        predicted_unders = np.sum(predicted_overs_and_unders == "Under")

        # Percentages for Over and Under accuracy, relative to each prediction type
        correct_over_percentage_relative = (correct_overs / predicted_overs) * 100 if predicted_overs > 0 else 0
        correct_under_percentage_relative = (correct_unders / predicted_unders) * 100 if predicted_unders > 0 else 0

        # Overall accuracy for each MAP level
        map_results = map_data["Result"].value_counts() / len(map_data)
        overall_accuracy = round(map_results.get("Correct", 0) * 100, 2)

        # Store results for the current MAP level
        results_by_map[map_level] = {
            "Total Predictions": len(map_data),
            "Overall Accuracy (%)": overall_accuracy,
            "Over Hit Rate (%)": round(correct_over_percentage_relative, 2),
            "Under Hit Rate (%)": round(correct_under_percentage_relative, 2),
        }

    # Calculate overall metrics across all MAP levels
    predicted_overs_and_unders_all = np.where(
        data["Model Prediction"] <= data["Line"], "Under", "Over"
    )
    actual_overs_and_unders_all = np.where(
        data["Actual"] <= data["Line"], "Under", "Over"
    )

    # Calculate correct Over and Under predictions for overall data
    correct_overs_all = np.sum((predicted_overs_and_unders_all == "Over") & (actual_overs_and_unders_all == "Over"))
    correct_unders_all = np.sum((predicted_overs_and_unders_all == "Under") & (actual_overs_and_unders_all == "Under"))

    # Predicted counts for overall data
    predicted_overs_all = np.sum(predicted_overs_and_unders_all == "Over")
    predicted_unders_all = np.sum(predicted_overs_and_unders_all == "Under")

    # Percentages for correct Over and Under predictions overall
    correct_over_percentage_relative_all = (correct_overs_all / predicted_overs_all) * 100 if predicted_overs_all > 0 else 0
    correct_under_percentage_relative_all = (correct_unders_all / predicted_unders_all) * 100 if predicted_unders_all > 0 else 0

    # Overall accuracy for the entire dataset
    overall_results = data["Result"].value_counts() / len(data)
    overall_accuracy_all = round(overall_results.get("Correct", 0) * 100, 2)

    # Store overall results
    results_by_map["Overall"] = {
        "Total Predictions": len(data),
        "Overall Accuracy (%)": overall_accuracy_all,
        "Over Hit Rate (%)": round(correct_over_percentage_relative_all, 2),
        "Under Hit Rate (%)": round(correct_under_percentage_relative_all, 2),
    }

    # Print the results
    for level, metrics in results_by_map.items():
        print(f"\nMAP Level: {level}")
        print(f"Total Predictions: {metrics['Total Predictions']}")
        print(f"Overall Accuracy: {metrics['Overall Accuracy (%)']}%")
        print(f"Over Hit Rate: {metrics['Over Hit Rate (%)']}%")
        print(f"Under Hit Rate: {metrics['Under Hit Rate (%)']}%")

    # Return data and results dictionary for further analysis if needed
    data = data.reset_index(drop=True)
    return data, results_by_map

In [12]:
print("Yesterday's Result")
yesterdays_hit_rate, results = results_by_days(data=prizepicks_hit_rate_df, n_days=1, specific_day=True)
yesterdays_hit_rate.to_csv("yesterdays_hit_rate.csv")

Yesterday's Result

MAP Level: MAPS 1
Total Predictions: 76
Overall Accuracy: 60.53%
Over Hit Rate: 57.38%
Under Hit Rate: 40.0%

MAP Level: MAPS 1-3
Total Predictions: 20
Overall Accuracy: 50.0%
Over Hit Rate: 40.0%
Under Hit Rate: 60.0%

MAP Level: MAPS 1-2
Total Predictions: 88
Overall Accuracy: 46.59%
Over Hit Rate: 42.31%
Under Hit Rate: 50.0%

MAP Level: MAPS 3
Total Predictions: 58
Overall Accuracy: 56.9%
Over Hit Rate: 73.08%
Under Hit Rate: 37.5%

MAP Level: Overall
Total Predictions: 242
Overall Accuracy: 53.72%
Over Hit Rate: 53.69%
Under Hit Rate: 45.16%


In [13]:
n_days = int(input("Enter the number of days you'd like to look back: "))
print(f"{n_days} Days Result")
n_days_hit_rate, results = results_by_days(data=prizepicks_hit_rate_df, n_days=n_days, specific_day=False)
n_days_hit_rate.to_csv(f"{n_days}day_hit_rate.csv")

30 Days Result

MAP Level: MAPS 3
Total Predictions: 803
Overall Accuracy: 50.93%
Over Hit Rate: 44.82%
Under Hit Rate: 54.73%

MAP Level: MAPS 1-2
Total Predictions: 1924
Overall Accuracy: 51.87%
Over Hit Rate: 48.33%
Under Hit Rate: 53.39%

MAP Level: MAPS 1-3
Total Predictions: 54
Overall Accuracy: 57.41%
Over Hit Rate: 46.67%
Under Hit Rate: 70.83%

MAP Level: MAPS 1
Total Predictions: 216
Overall Accuracy: 57.87%
Over Hit Rate: 52.17%
Under Hit Rate: 46.88%

MAP Level: Overall
Total Predictions: 2997
Overall Accuracy: 52.15%
Over Hit Rate: 47.66%
Under Hit Rate: 53.9%
